# Hatespeech classification pipeline

In [2]:
%load_ext autoreload
%autoreload 2

from hatedetection import HateSpeechAnalyzer

analyzer = HateSpeechAnalyzer("finiteautomata/beto-fine-grained-hatespeech-news", use_context=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
analyzer.predict("Amarillos roñosos de mierda, mátenlos", context="China reabre sus puertas")

HateClassificationOutput(hateful=True, calls_to_action=True, categories=['RACISM']

In [4]:
analyzer.predict("Judíos roñosos de mierda, mátenlos")

HateClassificationOutput(hateful=True, calls_to_action=True, categories=['RACISM', 'APPEARANCE']

In [5]:
analyzer.predict("Rusos de mierda, mátenlos")

HateClassificationOutput(hateful=True, calls_to_action=True, categories=['RACISM', 'CRIMINAL']

In [6]:
analyzer.predict("Hay que matar a todos las kukas")

HateClassificationOutput(hateful=True, calls_to_action=True, categories=['POLITICS']

In [7]:
analyzer.predict("Hay que matar a las cucarachas")

HateClassificationOutput(hateful=True, calls_to_action=True, categories=['WOMEN', 'POLITICS', 'CRIMINAL']

In [8]:
analyzer.predict("Maten a las feministas")

HateClassificationOutput(hateful=True, calls_to_action=False, categories=['WOMEN']

In [9]:
analyzer.predict("Un par de balazos a esos negros")

HateClassificationOutput(hateful=True, calls_to_action=True, categories=['RACISM', 'CRIMINAL']

In [10]:
analyzer.predict("Una pala para esos vagos")

HateClassificationOutput(hateful=True, calls_to_action=True, categories=['CLASS', 'POLITICS', 'CRIMINAL']

In [11]:
analyzer.predict("Mucha bala")

HateClassificationOutput(hateful=True, calls_to_action=False, categories=['CRIMINAL']

In [12]:
def show_context_and_comments(analyzer, context, comments):
    print(f"Contexto: {context}", "\n"*2)
    print("Comentarios:", "\n")
    
    for i, comment in enumerate(comments):
        print(f"{i:<3} -- {comment}")
        prediction = analyzer.predict(comment, context=context)
        if prediction.hateful:
            rep = "Discriminatorio"
            
            if prediction.calls_to_action:
                rep+= " y llama a la acción"
                
            rep += " contra " + ' - '.join(prediction.categories)
            
            print(rep)
        else:
            print("No discriminatorio")


In [13]:
show_context_and_comments(analyzer, 
    "Motín de presos en la cárcel de Devoto", 
    [
        "Negros chorros de mierda",
        "Este país está lleno de vagos",
        "Cuarenta ladrillos y esto se arregla...",
        "Bala",
        "Tres cohetes y unos cuantos menos",
        "Videla volvé!",
        "Ahhhn y donde estan las feminazis de mierda eh?",
        "Y qué van a decir los chinos sobre este desastre?",
        "Les parece bien esto?",
])

Contexto: Motín de presos en la cárcel de Devoto 


Comentarios: 

0   -- Negros chorros de mierda
Discriminatorio contra RACISM - CRIMINAL
1   -- Este país está lleno de vagos
Discriminatorio contra CLASS - CRIMINAL
2   -- Cuarenta ladrillos y esto se arregla...
Discriminatorio contra CRIMINAL
3   -- Bala
Discriminatorio y llama a la acción contra CRIMINAL
4   -- Tres cohetes y unos cuantos menos
Discriminatorio y llama a la acción contra CRIMINAL
5   -- Videla volvé!
Discriminatorio y llama a la acción contra CRIMINAL
6   -- Ahhhn y donde estan las feminazis de mierda eh?
Discriminatorio contra WOMEN - CRIMINAL
7   -- Y qué van a decir los chinos sobre este desastre?
Discriminatorio contra RACISM
8   -- Les parece bien esto?
No discriminatorio


In [14]:
show_context_and_comments(analyzer, 
    "Movilización de organizaciones sociales y piqueteros", 
    [
        "Negros de mierda",
        "Este país está lleno de vagos",
        "MATENLOS",
        "Bala",
        "El comunismo ha pervertido definitivamente a este país",
        "Tirás una pala y salen todos corriendo",
        "Laburar ni en pedo, no?",
        "No quieren laburar ni por casualidad",
        "Este país está lleno de choriplaneros",
        "Ahhhn y donde estan las feminazis de mierda eh?",
        "Y qué van a decir los chinos sobre este desastre?",
        "Nuevo orden mundial de Soros, el judaísmo y los chinos",
        "Les parece bien esto?",
])

Contexto: Movilización de organizaciones sociales y piqueteros 


Comentarios: 

0   -- Negros de mierda
Discriminatorio contra RACISM - CLASS
1   -- Este país está lleno de vagos
Discriminatorio contra CLASS
2   -- MATENLOS
Discriminatorio y llama a la acción contra CLASS - POLITICS
3   -- Bala
Discriminatorio y llama a la acción contra CLASS - POLITICS
4   -- El comunismo ha pervertido definitivamente a este país
Discriminatorio contra POLITICS
5   -- Tirás una pala y salen todos corriendo
Discriminatorio contra CLASS
6   -- Laburar ni en pedo, no?
Discriminatorio contra CLASS
7   -- No quieren laburar ni por casualidad
Discriminatorio contra CLASS
8   -- Este país está lleno de choriplaneros
Discriminatorio contra CLASS - POLITICS
9   -- Ahhhn y donde estan las feminazis de mierda eh?
Discriminatorio contra WOMEN - POLITICS
10  -- Y qué van a decir los chinos sobre este desastre?
No discriminatorio
11  -- Nuevo orden mundial de Soros, el judaísmo y los chinos
No discriminatorio
12  

In [15]:
context = "Plaga de langostas asola a la provincia de Tierra del Fuego"

comments = [
    "Qué asco!",
    "Matenlas!",
    "Ahhhn y donde estan las feministas de mierda eh?",
    "Y qué van a decir los chinos sobre este desastre?",
    "Cansado de este país",
    "Y los peronchos hacen algo?",
]

show_context_and_comments(analyzer, context, comments)

Contexto: Plaga de langostas asola a la provincia de Tierra del Fuego 


Comentarios: 

0   -- Qué asco!
No discriminatorio
1   -- Matenlas!
Discriminatorio y llama a la acción contra RACISM
2   -- Ahhhn y donde estan las feministas de mierda eh?
Discriminatorio contra WOMEN
3   -- Y qué van a decir los chinos sobre este desastre?
Discriminatorio contra RACISM
4   -- Cansado de este país
No discriminatorio
5   -- Y los peronchos hacen algo?
Discriminatorio contra POLITICS


In [16]:
show_context_and_comments(
    analyzer,
    "Romina C, una chica trans, nos cuenta sus vacaciones en Cancún",
    [
        "Asco",
        "Esta gente no teme a Dios",
        "Me parece que se llama Romino",
        "Es un camionero!",
        "Esta tal Romina que muestre el documento",
        "Romina es un flor de travesti",
        "Habría que ver si viene con paquete la tal Romina",
        "Romina es macho",
    ]
)

Contexto: Romina C, una chica trans, nos cuenta sus vacaciones en Cancún 


Comentarios: 

0   -- Asco
Discriminatorio contra WOMEN - LGBTI
1   -- Esta gente no teme a Dios
Discriminatorio contra WOMEN
2   -- Me parece que se llama Romino
No discriminatorio
3   -- Es un camionero!
No discriminatorio
4   -- Esta tal Romina que muestre el documento
Discriminatorio contra WOMEN
5   -- Romina es un flor de travesti
Discriminatorio contra WOMEN - LGBTI
6   -- Habría que ver si viene con paquete la tal Romina
Discriminatorio contra WOMEN
7   -- Romina es macho
Discriminatorio contra WOMEN


In [17]:
show_context_and_comments(
    analyzer,
    "Florencia de la V presenta su nuevo novio en sociedad",
    [
        "Asco",
        "Esta gente no teme a Dios",
        "Me parece que se llama Romino",
        "Es un camionero!",
        "Esta tal Romina que muestre el documento",
        "Romina es un flor de travesti",
        "Habría que ver si viene con paquete la tal Romina",
        "Florcita tiene algo entre las piernas",
        "Flor es un macho",
        "Naciste hombre, Carlos",
        "Viene con dos pelotas entre las piernas eh!",
        "Florencio. Florencio. Mostrá tu DNI para que sepamos tu nombre",
    ]
)

Contexto: Florencia de la V presenta su nuevo novio en sociedad 


Comentarios: 

0   -- Asco
Discriminatorio contra WOMEN
1   -- Esta gente no teme a Dios
Discriminatorio contra WOMEN
2   -- Me parece que se llama Romino
No discriminatorio
3   -- Es un camionero!
No discriminatorio
4   -- Esta tal Romina que muestre el documento
No discriminatorio
5   -- Romina es un flor de travesti
Discriminatorio contra WOMEN - LGBTI
6   -- Habría que ver si viene con paquete la tal Romina
No discriminatorio
7   -- Florcita tiene algo entre las piernas
Discriminatorio contra WOMEN
8   -- Flor es un macho
Discriminatorio contra LGBTI
9   -- Naciste hombre, Carlos
Discriminatorio contra LGBTI
10  -- Viene con dos pelotas entre las piernas eh!
Discriminatorio contra WOMEN
11  -- Florencio. Florencio. Mostrá tu DNI para que sepamos tu nombre
No discriminatorio


In [18]:
show_context_and_comments(
    analyzer,
    "Liverpool y Tottenham empataron 0 a 0 en la final de la Champions League",
    [
        "Asco",
        "Esta gente no teme a Dios",
        "Dos degenerados",
        "Qué hijos de puta",
        "Lamela jugó?",
        
    ]
)

Contexto: Liverpool y Tottenham empataron 0 a 0 en la final de la Champions League 


Comentarios: 

0   -- Asco
No discriminatorio
1   -- Esta gente no teme a Dios
No discriminatorio
2   -- Dos degenerados
No discriminatorio
3   -- Qué hijos de puta
No discriminatorio
4   -- Lamela jugó?
No discriminatorio


In [19]:
show_context_and_comments(
    analyzer,
    "Roberto y Juan Pérez se casan la semana que viene",
    [
        "Asco",
        "Esta gente no teme a Dios",
        "Dos degenerados",
        "Qué hijos de puta",
        "Inmundos",
    ]
)

Contexto: Roberto y Juan Pérez se casan la semana que viene 


Comentarios: 

0   -- Asco
No discriminatorio
1   -- Esta gente no teme a Dios
No discriminatorio
2   -- Dos degenerados
No discriminatorio
3   -- Qué hijos de puta
No discriminatorio
4   -- Inmundos
No discriminatorio
